In [ ]:
#podział na odchylenia wszystkie i odchylenia złe, bez odchyleń

#pca
#miernik na podstawie ostatniej wartości
#użycie wag do LDA w czasie

#granica LDA

#50 pacjentów zmarłych i 50 zywych

In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import plotly.express as px
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.metrics import confusion_matrix
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestClassifier
import random
import plotly.graph_objs as go

### dane

In [3]:
df_prep = pd.read_csv("..\chapter3\gasometry_prepared_data.csv")
cols=["BETET",	"CO2TET",	"HCO3ACTE",	"HCO3STTE",	"O2SATTET",	"O2TET", "IONH"]
my_blue="#0064B2";
my_red="#D61600";

In [131]:
random.seed(42)  
df_zgon = df_prep.groupby('PACJENT_NR').last()
sample_zgon = random.sample(list(df_zgon[df_zgon['ZGON'] ==1 ].index), 40)
sample_nzgon = random.sample(list(df_zgon[df_zgon['ZGON'] == 0].index), 40)
indexes=[*sample_zgon, *sample_nzgon]
pacjenci_80 = df_zgon.loc[df_zgon.index.isin(indexes)].index
data80=df_prep.loc[df_prep["PACJENT_NR"].isin(pacjenci_80)].reset_index(drop=True)
data27=df_prep.loc[~df_prep["PACJENT_NR"].isin(pacjenci_80)].reset_index(drop=True)

In [128]:
data80

,Unnamed: 0,PACJENT_NR,BADANIE_NR,BETET,CO2TET,HCO3ACTE,HCO3STTE,O2SATTET,O2TET,IONH,ZGON
0,60,3,1,0.478261,1.42,0.883333,0.750,0.74,2.148,1.008167,0.0
1,61,3,2,1.782609,2.06,1.700000,1.675,0.90,3.924,0.795364,0.0
2,62,3,3,1.804348,1.24,1.716667,2.050,0.92,3.984,0.147539,0.0
3,63,3,4,2.826087,2.18,2.383333,2.600,0.56,0.936,0.381504,0.0
4,64,3,5,2.652174,2.31,2.633333,2.875,0.76,1.708,0.301894,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2165,2929,145,5,-0.717391,1.63,0.133333,-0.350,0.18,0.420,2.294287,1.0
2166,2930,145,6,-0.891304,1.39,-0.016667,-0.450,0.76,1.640,2.254399,1.0
2167,2931,145,7,-1.065217,1.71,-0.050000,-0.650,-0.06,0.316,2.708690,1.0
2168,2932,145,8,-0.695652,1.15,0.066667,-0.275,0.62,1.600,1.857967,1.0


In [114]:
cols=data80.columns[3:-1]

### PCA

In [136]:
X_data=data80[cols].reset_index(drop=True)

pca = PCA()
components = pca.fit_transform(X_data)

pca.explained_variance_ratio_


array([0.45236006, 0.27342346, 0.20820373, 0.05506843, 0.00674966,
       0.00301134, 0.00118333])

In [116]:
# Wyjaśniona wariancja
explained_variance_ratio = pca.explained_variance_ratio_

# Kumulatywna suma wyjaśnionej wariancji
cumulative_explained_variance_ratio = explained_variance_ratio.cumsum()

# Tworzenie osypiska (scree plot) za pomocą Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, len(explained_variance_ratio) + 1)), y=explained_variance_ratio,
                         mode='lines+markers', name='Explained Variance Ratio'))

# Konfiguracja osi i tytułu
fig.update_layout(title='Scree Plot - Explained Variance Ratio',
                  xaxis_title='Principal Component',
                  yaxis_title='Explained Variance Ratio')

# Wyświetlenie wykresu
fig.show()


In [137]:
X_data=data80[cols].reset_index(drop=True)
y_data=data80['ZGON']

pca = PCA(n_components=3)
components = pca.fit_transform(X_data)
components_test=pca.transform(data27[cols])

pca.explained_variance_ratio_

array([0.45236006, 0.27342346, 0.20820373])

In [118]:
# Wagi (współczynniki) PCA
weights_pca = pca.components_
weights_df = pd.DataFrame(weights_pca, columns=X_data.columns)
weights_df 

,BETET,CO2TET,HCO3ACTE,HCO3STTE,O2SATTET,O2TET,IONH
0,-0.597659,-0.096732,-0.412235,-0.550879,0.064767,0.254929,0.301425
1,-0.118911,0.314074,-0.007447,-0.135389,-0.585908,-0.662740,0.293801
2,0.092540,0.743590,0.236178,0.038823,0.154694,0.389459,0.453447


In [129]:
transformed_df = pd.DataFrame(components, columns=[f'PC{i+1}' for i in range(components.shape[1])])
transformed_df[['PACJENT_NR', 'BADANIE_NR', 'ZGON']] = data80[['PACJENT_NR', 'BADANIE_NR', 'ZGON']]
transformed_df

,PC1,PC2,PC3,PACJENT_NR,BADANIE_NR,ZGON
0,1.748440,-0.766392,0.228144,3,1,0.0
1,0.459727,-2.185096,1.673471,3,2,0.0
2,0.133925,-2.737931,0.816941,3,3,0.0
3,-1.875280,-0.343929,0.652603,3,4,0.0
4,-1.852702,-0.973719,1.098398,3,5,0.0
...,...,...,...,...,...,...
2165,3.268744,1.447440,-0.122612,145,5,1.0
2166,3.849379,0.247309,0.190299,145,6,1.0
2167,3.792582,1.887203,-0.039979,145,7,1.0
2168,3.486145,0.116417,-0.160578,145,8,1.0


In [138]:
transformed_test_df = pd.DataFrame(components_test, columns=[f'PC{i+1}' for i in range(components.shape[1])])
transformed_test_df[['PACJENT_NR', 'BADANIE_NR', 'ZGON']] = data27[['PACJENT_NR', 'BADANIE_NR', 'ZGON']]
transformed_test_df

,PC1,PC2,PC3,PACJENT_NR,BADANIE_NR,ZGON
0,1.389841,3.615450,-0.445285,1,1,0.0
1,3.318768,-0.660209,5.112773,1,2,0.0
2,2.591719,-2.589547,3.661287,1,3,0.0
3,1.344556,0.408562,1.443207,1,4,0.0
4,1.948062,-0.602604,1.873868,1,5,0.0
...,...,...,...,...,...,...
759,-2.095276,1.556205,-0.756072,140,20,1.0
760,-2.018273,-0.018412,0.093890,140,21,1.0
761,-2.613907,0.289425,-0.619470,140,22,1.0
762,-2.445426,0.935426,-0.755629,140,23,1.0


### LDA z ostatnich wartości

In [142]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix

In [144]:
X_train_data_lda=transformed_df.groupby("PACJENT_NR").last()
X_test_data_lda=transformed_test_df.groupby("PACJENT_NR").last()

In [145]:
# Definiowanie zmiennych niezależnych i celu
X_train = X_train_data_lda.drop(columns="ZGON")
y_train = X_train_data_lda['ZGON']

X_test = X_test_data_lda.drop(columns="ZGON")
y_test = X_test_data_lda['ZGON']

# Tworzenie i trenowanie modelu LDA
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

# Predykcje na zbiorze testowym
y_pred = lda.predict(X_test)

# Ocena modelu
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.5925925925925926
Confusion Matrix:
[[10  2]
 [ 9  6]]
